In [38]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [35]:
# 1. 기존 numpy 제거
!pip uninstall numpy
# 2. 호환되는 numpy 버전 설치
!pip install numpy==1.24.3
# 3. 필요 시 scipy 재설치:
!pip install catboost

^C
Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
     ---------- ----------------------------- 2.9/10.9 MB 16.8 MB/s eta 0:00:01
     --------------------------------------  10.7/10.9 MB 28.0 MB/s eta 0:00:01
     --------------------------------------- 10.9/10.9 MB 27.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\cli\req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2032.0_x64__qbz5n2kfra8p0\Lib\site-packages\pip\_internal\commands\install.py", line 379, in run
    requirement_set = resolver

Defaulting to user installation because normal site-packages is not writeable


In [19]:
train = pd.read_csv("train.csv", index_col='id')

In [20]:
#df = pd.concat([train, test])
# to_shuffle = ["loantoincome", "loan_percent_incometoincome", "cb_person_cred_hist_lengthtoincome", 'person_age_to_person_income', 
#               'person_emp_length_to_cb_person_cred_hist_length', 'person_emp_length_to_person_age', 'loan_int_rate_to_loan_amnt',
#               "person_home_ownership_income", "loan_intent_income", "loan_grade_income"]

df = train

# featrue engineering
df["loantoincome"] = ((df["loan_amnt"] / df["person_income"])).astype(str).astype('category')
df["loan_percent_incometoincome"] = ((df["loan_percent_income"] / df["person_income"])).astype(str).astype('category')
# df["cb_person_cred_hist_lengthtoincome"] = ((df["cb_person_cred_hist_length"] / df["person_income"])).astype(str).astype('category')
df['person_age_to_person_income'] = (df['person_age'] / df['person_income']).astype(str).astype('category')
# df['person_emp_length_to_cb_person_cred_hist_length'] = (df['person_emp_length'] / df['cb_person_cred_hist_length']).astype(str).astype('category')
df['person_emp_length_to_person_age'] = (df['person_emp_length'] / df['person_age']).astype(str).astype('category')
df['loan_int_rate_to_loan_amnt'] = (df['loan_int_rate'] / df['loan_amnt']).astype(str).astype('category')

df['person_home_ownership'] = df['person_home_ownership'].replace({'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'OTHER': 3}).astype('category')
df['loan_intent'] = df['loan_intent'].replace({'EDUCATION': 0, 'MEDICAL': 1, 'PERSONAL': 2, 'VENTURE': 3, 'DEBTCONSOLIDATION': 4, 'HOMEIMPROVEMENT': 5}).astype('category')
df['loan_grade'] = df['loan_grade'].replace({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6}).astype('category')
df['cb_person_default_on_file'] = df['cb_person_default_on_file'].replace({'N': 0, 'Y': 1}).astype('category')
df['person_emp_length'] = df['person_emp_length'].astype(str).astype('category')
df['loan_int_rate'] = (df['loan_int_rate'] * 100).astype(str).astype('category')
df['loan_percent_income'] = (df['loan_percent_income'] * 100).astype(str).astype('category')

# more feature engineering
df["person_home_ownership_income"] = pd.Series(pd.factorize((df["person_home_ownership"].astype(str) + df["person_income"].astype(str)).to_numpy())[0]).astype('category')
# df["loan_intent_income"] = pd.Series(pd.factorize((df["loan_intent"].astype(str) + df["person_income"].astype(str)).to_numpy())[0]).astype('category')
# df["loan_grade_income"] = pd.Series(pd.factorize((df["loan_grade"].astype(str) + df["person_income"].astype(str)).to_numpy())[0]).sample(frac=1).reset_index(drop=True).astype('category')

# Shuffle
# for col in to_shuffle:
#     add_shuffled(df, col)

train = df[:train.shape[0]]
test = df[train.shape[0]:]

train

C:\Users\PC\AppData\Local\Temp\ipykernel_45216\2903163201.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['person_home_ownership'] = df['person_home_ownership'].replace({'RENT': 0, 'MORTGAGE': 1, 'OWN': 2, 'OTHER': 3}).astype('category')
C:\Users\PC\AppData\Local\Temp\ipykernel_45216\2903163201.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['loan_intent'] = df['loan_intent'].replace({'EDUCATION': 0, 'MEDICAL': 1, 'PERSONAL': 2, 'VENTURE': 3, 'DEBTCONSOLIDATION': 4, 'HOMEIMPROVEMENT': 5}).astype('category')
C:\User

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,loantoincome,loan_percent_incometoincome,person_age_to_person_income,person_emp_length_to_person_age,loan_int_rate_to_loan_amnt,person_home_ownership_income
id,,,,,,,,,,,,,,,,,,
0,37,35000,0,0.0,0,1,6000,1149.0,17.0,0,14,0,0.17142857142857143,4.857142857142858e-06,0.0010571428571428572,0.0,0.001915,0
1,22,56000,2,6.0,1,2,4000,1335.0,7.000000000000001,0,2,0,0.07142857142857142,1.25e-06,0.0003928571428571429,0.2727272727272727,0.0033374999999999998,1
2,29,28800,2,8.0,2,0,6000,890.0,21.0,0,10,0,0.20833333333333334,7.291666666666667e-06,0.0010069444444444444,0.27586206896551724,0.0014833333333333335,2
3,30,70000,0,14.0,3,1,12000,1111.0,17.0,0,5,0,0.17142857142857143,2.428571428571429e-06,0.00042857142857142855,0.4666666666666667,0.0009258333333333333,3
4,22,60000,0,2.0,1,0,6000,692.0,10.0,0,3,0,0.1,1.6666666666666667e-06,0.00036666666666666667,0.09090909090909091,0.0011533333333333333,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,34,120000,1,5.0,0,3,25000,1595.0,21.0,1,10,0,0.20833333333333334,1.75e-06,0.00028333333333333335,0.14705882352941177,0.000638,30
58641,28,28800,0,0.0,1,2,10000,1273.0,35.0,0,8,1,0.3472222222222222,1.2152777777777777e-05,0.0009722222222222222,0.0,0.001273,171
58642,23,44000,0,7.0,0,3,6800,1600.0,15.0,0,2,1,0.15454545454545454,3.409090909090909e-06,0.0005227272727272727,0.30434782608695654,0.002352941176470588,157


In [23]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


In [21]:
## train model
aucs = []
preds = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, valid_idx) in enumerate(skf.split(train, train['loan_status'])):
    print(f'### Fold {fold+1} Training ###')

    X_train = train.loc[train_idx, [c for c in train.columns if c not in ['id', 'loan_status']]]
    y_train = train.loc[train_idx, 'loan_status']
    X_valid = train.loc[valid_idx, X_train.columns]
    y_valid = train.loc[valid_idx, 'loan_status']
    X_test = test[X_train.columns]

    X_train_pool = Pool(X_train, y_train, cat_features=X_train.columns.values)
    X_valid_pool = Pool(X_valid, y_valid, cat_features=X_valid.columns.values)
    X_test_pool = Pool(X_test, cat_features=X_test.columns.values)

    model = CatBoostClassifier(
        loss_function='Logloss',
        eval_metric='AUC',
        learning_rate=0.99,       # learning rate used for the gradient step
        iterations=150,          # max number of trees 
        depth=6,                  # Depth of the trees
        random_strength=0,        # Randomness to use for scoring splits when tree is selected. (Avoid overfitting)
        l2_leaf_reg=200,            # regularization
        task_type='GPU',
        random_seed=42,
        verbose=False,
        
        # HPO from winning insurance approach
        leaf_estimation_method="Newton",
        bootstrap_type="Bernoulli",
        grow_policy="SymmetricTree",
        early_stopping_rounds=10,
        use_best_model=True,
        score_function='NewtonL2',
        leaf_estimation_iterations=1,
        subsample=1,
#         reg_lambda=0.5,
        od_pval=0.1
    )

    model.fit(X=X_train_pool, eval_set=X_valid_pool, verbose=1000,)
    
    pred_train = model.predict_proba(X_train_pool)[:, 1]
    pred_valid = model.predict_proba(X_valid_pool)[:, 1]
    preds.append(model.predict_proba(X_test_pool)[:, 1])

    auc = roc_auc_score(y_valid, pred_valid)
    auc_train = roc_auc_score(y_train, pred_train)
    aucs.append(auc)

    print(f'Fold {fold+1} AUC: {auc:.5f}, Training_AUC: {auc_train:5f}\n')

print(f'\nOverall AUC: {np.mean(aucs):.5f} +/- {np.std(aucs):.5f}')

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject